In [1]:
import numpy as np
import pandas as pd
import re
import glob
from scipy.misc import comb
from imagery_psychophysics.src.stirling_maps import stirling_num_of_2nd_kind as snk
from time import time
from imagery_psychophysics.src.inference_machinery import *
from imagery_psychophysics.src.counting_machinery import *


%matplotlib inline


#### Read data

In [3]:
df = pd.read_pickle('/media/tnaselar/Data/scratch/z1_KL.pkl')

In [4]:
df.head()

,image,image_on,probe,resp_on,response,state,subj
0,candle_01,117.5410,11_12_21,127.8745,3,img,AS
1,candle_01,127.9244,15_7_16,139.0581,2,img,AS
2,candle_01,139.1078,32_24_33,151.4748,2,img,AS
3,candle_01,151.5415,37,156.2748,2,img,AS
4,candle_01,156.3414,18_10_19,163.0749,2,img,AS


In [5]:
df[['image','subj']].drop_duplicates()

,image,subj
0,candle_01,AS
690,peaches_05,AS
1380,candle_01,z1
2070,peaches_05,z1
2760,candle_01,KL
3450,peaches_05,KL
4140,cheeseburger_03,KL
4830,grape_juice_02,KL


In [6]:
##split up, package response dicts
n_colors = 10
n_probes = df[['probe']].drop_duplicates().apply(lambda x: np.max(map(int,x.str.split('_')[0])),axis=1).max()+1
target_sequence = range(1,n_probes+1)


In [7]:
image_grp = df.groupby(by=['image','subj', 'state'])

re_ass_sults = []
for name,grp in image_grp:
    print name
    non_zero = 0
    resp_df = grp[['response']].copy()
    resp_df['probe'] = grp[['probe']].apply(lambda x: tuple(map(int,x.str.split('_')[0])),axis=1)
    experimental_observations = dict(zip(resp_df.probe.values,resp_df.response.values))
    exp_obs = format_experimental_obsevations(experimental_observations)
#     seq = create_sequence_of_nbd_obs(target_sequence,exp_obs.copy())
    counter_seq = []
    for ii in target_sequence:
#         print ii
        if not np.mod(ii,9):
            continue
        else:
            nbd_windows = window([jj for jj in [ii,ii-1,ii+8,ii+9] if jj < n_probes])
#         print nbd_windows.string
        nbd_powerset = nbd_windows.powerset(nonempty=True)
        counter = consistent_map_counter(ii,nbd_powerset)
        nbd_color_counts = []    
        for tup in nbd_powerset.tups:
            nbd_color_counts.append(exp_obs[tup])
        if not any(np.isnan(nbd_color_counts)):
            non_zero += bool(counter.count_consistent_coloring(n_colors,nbd_color_counts))
    re_ass_sults.append(name+(non_zero,))

('candle_01', 'AS', 'img')
('candle_01', 'AS', 'pcp')
('candle_01', 'KL', 'img')
('candle_01', 'KL', 'pcp')
('candle_01', 'z1', 'img')
('candle_01', 'z1', 'pcp')
('cheeseburger_03', 'KL', 'img')
('cheeseburger_03', 'KL', 'pcp')
('grape_juice_02', 'KL', 'img')
('grape_juice_02', 'KL', 'pcp')
('peaches_05', 'AS', 'img')
('peaches_05', 'AS', 'pcp')
('peaches_05', 'KL', 'img')
('peaches_05', 'KL', 'pcp')
('peaches_05', 'z1', 'img')
('peaches_05', 'z1', 'pcp')


In [8]:
results = pd.DataFrame(data=re_ass_sults, columns = ['image','subj','state','non_zeros'])

In [9]:
results

,image,subj,state,non_zeros
0,candle_01,AS,img,14
1,candle_01,AS,pcp,34
2,candle_01,KL,img,33
3,candle_01,KL,pcp,40
4,candle_01,z1,img,11
5,candle_01,z1,pcp,30
6,cheeseburger_03,KL,img,9
7,cheeseburger_03,KL,pcp,31
8,grape_juice_02,KL,img,14
9,grape_juice_02,KL,pcp,33


In [ ]:
results.groupby(['subj', 'state'])

In [ ]:
7/13.

In [ ]:
results.groupby(by=['image', 'subj']).apply(lambda row: row[['non_zeros']].ix[0]/row[['non_zeros']].ix[1])

In [ ]:
sub_state = 
for name,grp in sub_state:
    for stim in grp.image.drop_duplicates():
        ratio = grp[grp.image==]['non']

In [ ]:
plt.plot([0,1,2,3],[33/40., 14./33, 9/31., 7/34.])
plt.plot([0,3],[11/30., 8/11.],hold=True)